<a href="https://colab.research.google.com/github/eaniket/InceptionV3/blob/master/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

C:\Users\us\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\us\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

ModuleNotFoundError: No module named 'google.colab'

In [0]:
nb_classes = 5  # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 299, 299  # change based on the shape/structure of your images
batch_size = 32  
nb_epoch = 50  
learn_rate = 1e-4  
momentum = .9 
transformation_ratio = .05  
nb_train_samples = 3682  
nb_validation_samples = 584  

In [0]:
def train(train_data_dir, validation_data_dir, model_path):
    
    
    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
   
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)

    # # Top Model Block    
    x = base_model.output
    x = GlobalAveragePooling2D((x)
    
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(nb_classes, activation='softmax', name='predictions')(x)

  
    model = Model(base_model.input, predictions)
    print(model.summary())
    
    for layer in base_model.layers:
        layer.trainable = False

    # Data augmentation 
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1. / 255)

  
    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=[img_width, img_height],
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=[img_width, img_height],
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')
    # Model compiling
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

    # save weights of best training epoch

    top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
    
    from PIL import ImageFile
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    # Train Simple CNN
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch / 4,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size
                     
                        )
    return model
    

In [0]:
train_path = 'gdrive/My Drive/image_dataset/data_set/'
test_path = 'gdrive/My Drive/testdata/'

In [0]:
#train_path.head()

In [22]:
model = train(train_path, test_path, "gdrive/My Drive/")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 149, 149, 32) 96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 149, 149, 32) 0           batch_normalization_377[0][0]    
__________________________________________________________________________________________________
conv2d_378

W0630 08:41:57.341848 139844732229504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0630 08:41:57.488650 139844732229504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 3987 images belonging to 5 classes.
Epoch 1/12


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


115/115 [==============================] - 1530s 13s/step - loss: 0.9191 - acc: 0.6891 - val_loss: 0.2681 - val_acc: 0.8924
Epoch 2/12
115/115 [==============================] - 1249s 11s/step - loss: 0.4751 - acc: 0.8315 - val_loss: 0.2810 - val_acc: 0.8889
Epoch 3/12
115/115 [==============================] - 1330s 12s/step - loss: 0.4090 - acc: 0.8518 - val_loss: 0.1967 - val_acc: 0.9219
Epoch 4/12
115/115 [==============================] - 1350s 12s/step - loss: 0.4047 - acc: 0.8552 - val_loss: 0.1422 - val_acc: 0.9497
Epoch 5/12
115/115 [==============================] - 1364s 12s/step - loss: 0.3727 - acc: 0.8670 - val_loss: 0.2222 - val_acc: 0.9080
Epoch 6/12
115/115 [==============================] - 1370s 12s/step - loss: 0.3407 - acc: 0.8739 - val_loss: 0.1027 - val_acc: 0.9740
Epoch 7/12
115/115 [==============================] - 1356s 12s/step - loss: 0.3254 - acc: 0.8812 - val_loss: 0.2210 - val_acc: 0.9165
Epoch 8/12
115/115 [==============================] - 1356s 12s/st

In [0]:
model.save("gdrive/My Drive/inception.h5")

In [24]:
layer_name = "mixed10"
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print(intermediate_layer_model)

In [26]:
li=[]
img_path = "gdrive/My Drive/image_dataset/data_set/shoes/00000302.jpg"            
image2 = image.load_img(img_path, target_size=(299,299))
image2 = image.img_to_array(image2)
print(image2.shape)
image2 = np.expand_dims(image2, axis=0)
print(image2.shape)
image2 = preprocess_input(image2)
intermediate_output = intermediate_layer_model.predict(image2)
print(intermediate_output)
li.append(intermediate_output)
print(li)

(299, 299, 3)
(1, 299, 299, 3)
[[[[0.00000000e+00 4.31867003e-01 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 1.26274526e-01 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [1.54062402e+00 4.78087544e-01 0.00000000e+00 ... 0.00000000e+00
    7.73721933e-01 2.30732709e-01]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 2.23332345e-01 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [4.26317501e+00 0.00000

In [27]:
#for output, change this image path


img_path = "gdrive/My Drive/image_dataset/data_set/shoes/00000302.jpg"
            
image2 = image.load_img(img_path, target_size=(299,299))
image2 = image.img_to_array(image2)
print(image2.shape)
image2 = np.expand_dims(image2, axis=0)
print(image2.shape)
image2 = preprocess_input(image2)
output = model.predict(image2)
print(output[0])
lis = list(output[0])
out = (max(output[0]))
print(out)
l = lis.index(out)
print(l)
if l == 0:
  print("its a dress")
  #auto encoder for dress
elif l == 1:
   print("its a shirt")
  #autoencoder for shirt
elif l == 2 :
   print("its a shoe")
  #autoencoder for shoe
else:
   print("its a short")
  # autoencoder for short

(299, 299, 3)
(1, 299, 299, 3)
[1.48195602e-06 8.14806526e-07 1.13706647e-05 9.99975204e-01
 1.10727315e-05]
0.9999752
3
its a short
